# W_GL_SEGMENT_D ETL Process
### SCD Type 2 Dimension - Staging to Dimension with Effective Dating

In [ ]:
%sql
-- 1) PARAMETERS
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID, 2625027 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS,
  '1899-01-01 00:00:00' AS LOW_DATE, '3714-01-01 00:00:00' AS HI_DT
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SILOS_SIL_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING WITH LOOKUPS
CREATE OR REPLACE TEMP VIEW stg_segment_with_lookups AS
SELECT
  ds.SEGMENT_LOV_ID, COALESCE(ds.SEGMENT_VAL_CODE, '__NOT_APPLICABLE__') AS SEGMENT_VAL_CODE,
  COALESCE(ucr.ROW_WID, 0) AS CREATED_BY_WID, COALESCE(uch.ROW_WID, 0) AS CHANGED_BY_WID,
  ds.CREATED_ON_DT, ds.CHANGED_ON_DT, ds.AUX1_CHANGED_ON_DT, ds.AUX2_CHANGED_ON_DT,
  ds.AUX3_CHANGED_ON_DT, ds.AUX4_CHANGED_ON_DT, ds.SRC_EFF_FROM_DT, ds.SRC_EFF_TO_DT,
  CASE WHEN ds.DELETE_FLG = 'Y' THEN 'Y' ELSE 'N' END AS DELETE_FLG,
  ds.DATASOURCE_NUM_ID, ds.INTEGRATION_ID, ds.SET_ID, ds.TENANT_ID, ds.X_CUSTOM,
  COALESCE(ds.SRC_EFF_FROM_DT, to_timestamp('1899-01-01')) AS EFFECTIVE_FROM_DT,
  to_timestamp('3714-01-01') AS EFFECTIVE_TO_DT
FROM workspace.oracle_edw.w_gl_segment_ds ds
LEFT JOIN workspace.oracle_edw.w_user_d ucr
  ON ds.DATASOURCE_NUM_ID = ucr.DATASOURCE_NUM_ID AND ds.CREATED_BY_ID = ucr.INTEGRATION_ID
  AND ds.CHANGED_ON_DT >= ucr.EFFECTIVE_FROM_DT AND ds.CHANGED_ON_DT < ucr.EFFECTIVE_TO_DT
  AND ucr.DELETE_FLG = 'N'
LEFT JOIN workspace.oracle_edw.w_user_d uch
  ON ds.DATASOURCE_NUM_ID = uch.DATASOURCE_NUM_ID AND ds.CHANGED_BY_ID = uch.INTEGRATION_ID
  AND ds.CHANGED_ON_DT >= uch.EFFECTIVE_FROM_DT AND ds.CHANGED_ON_DT < uch.EFFECTIVE_TO_DT
  AND uch.DELETE_FLG = 'N';

In [ ]:
%sql
-- 3) CHANGE DETECTION
CREATE OR REPLACE TEMP VIEW stg_segment_with_action AS
SELECT s.*, 'I' AS IND_UPDATE, 'Y' AS IND_CURRENT_FLG
FROM stg_segment_with_lookups s
WHERE NOT EXISTS (
  SELECT 1 FROM workspace.oracle_edw.w_gl_segment_d t
  WHERE t.SRC_EFF_FROM_DT = s.SRC_EFF_FROM_DT
    AND t.DATASOURCE_NUM_ID = s.DATASOURCE_NUM_ID
    AND t.INTEGRATION_ID = s.INTEGRATION_ID
    AND (COALESCE(t.CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')))
    AND (COALESCE(t.AUX1_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')) = COALESCE(s.AUX1_CHANGED_ON_DT, to_date('3714-01-01', 'yyyy-MM-dd')))
);

In [ ]:
%sql
-- 4) INSERT NEW RECORDS
INSERT INTO workspace.oracle_edw.w_gl_segment_d
(
  SEGMENT_LOV_ID, SEGMENT_VAL_CODE, CREATED_BY_WID, CHANGED_BY_WID,
  CREATED_ON_DT, CHANGED_ON_DT, AUX1_CHANGED_ON_DT, AUX2_CHANGED_ON_DT, AUX3_CHANGED_ON_DT, AUX4_CHANGED_ON_DT,
  SRC_EFF_FROM_DT, SRC_EFF_TO_DT, DELETE_FLG, DATASOURCE_NUM_ID, INTEGRATION_ID,
  SET_ID, TENANT_ID, X_CUSTOM, EFFECTIVE_FROM_DT, EFFECTIVE_TO_DT, CURRENT_FLG,
  W_INSERT_DT, W_UPDATE_DT, ETL_PROC_WID, SCD1_WID
)
SELECT s.SEGMENT_LOV_ID, s.SEGMENT_VAL_CODE, s.CREATED_BY_WID, s.CHANGED_BY_WID,
  s.CREATED_ON_DT, s.CHANGED_ON_DT, s.AUX1_CHANGED_ON_DT, s.AUX2_CHANGED_ON_DT, s.AUX3_CHANGED_ON_DT, s.AUX4_CHANGED_ON_DT,
  s.SRC_EFF_FROM_DT, s.SRC_EFF_TO_DT, s.DELETE_FLG, s.DATASOURCE_NUM_ID, s.INTEGRATION_ID,
  s.SET_ID, s.TENANT_ID, s.X_CUSTOM, s.EFFECTIVE_FROM_DT, s.EFFECTIVE_TO_DT, s.IND_CURRENT_FLG,
  current_timestamp(), current_timestamp(), p.ETL_PROC_WID, 0
FROM stg_segment_with_action s
CROSS JOIN etl_params p
WHERE s.IND_UPDATE = 'I';

In [ ]:
%sql
-- 5) UPDATE CONTROL AND LOG TABLES (Similar to previous notebooks)
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SILOS_SIL_GLSEGMENTDIMENSION' AS package_name, 'W_GL_SEGMENT_D' AS target_table_name,
    p.ETL_USAGE_CODE, p.ETL_PROC_WID, p.EXECUTION_ID AS load_plan_id, current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.etl_load_date = src.etl_load_date, tgt.committed = src.committed
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name,
  src.etl_usage_code, src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
-- 6) CLEANUP
DROP VIEW IF EXISTS stg_segment_with_action;
DROP VIEW IF EXISTS stg_segment_with_lookups;
DROP VIEW IF EXISTS etl_params;